# Suspicious Use of Procdump
Detects suspicious uses of the SysInternals Procdump utility by using a special command line parameter in combination with the lsass.exe process. This way we're also able to catch cases in which the attacker has renamed the procdump executable.

## Rule Content
```
- title: Suspicious Use of Procdump
  id: 5afee48e-67dd-4e03-a783-f74259dcf998
  description: Detects suspicious uses of the SysInternals Procdump utility by using
    a special command line parameter in combination with the lsass.exe process. This
    way we're also able to catch cases in which the attacker has renamed the procdump
    executable.
  status: experimental
  references:
  - Internal Research
  author: Florian Roth
  date: 2018/10/30
  modified: 2019/10/14
  tags:
  - attack.defense_evasion
  - attack.t1036
  - attack.credential_access
  - attack.t1003
  - car.2013-05-009
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection1:
      CommandLine:
      - '* -ma *'
    selection2:
      CommandLine:
      - '* lsass*'
    selection3:
      CommandLine:
      - '* -ma ls*'
    condition: ( selection1 and selection2 ) or selection3
  falsepositives:
  - Unlikely, because no one should dump an lsass process memory
  - Another tool that uses the command line switches of Procdump
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_command_line.keyword:(*\ \-ma\ *) AND process_command_line.keyword:(*\ lsass*)) OR process_command_line.keyword:(*\ \-ma\ ls*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()